# Etapa 5 – Apoio à Avaliação da IA

<p align="center">
    <img src="https://img.icons8.com/ios-filled/100/artificial-intelligence.png" width="80" alt="Ícone IA"/>
</p>
Subetapa: Filtragem dos Parágrafos para Geração de Perguntas

<img src="https://img.icons8.com/ios-filled/50/000000/right--v1.png" width="24" style="vertical-align:middle; margin-right:8px;"/> Objetivo da Subetapa
Durante a preparação do corpus para a geração automática de perguntas, observou-se que alguns arquivos .txt, mesmo após as etapas anteriores de limpeza, ainda continham trechos irrelevantes. Esses trechos, quando enviados ao modelo de NLP, causavam lentidão no processamento e geravam perguntas mal formuladas, com baixa utilidade para fins de validação.

Para resolver esse gargalo, foi desenvolvido um script de filtragem semântica automatizada. Ele identifica e remove parágrafos com conteúdo não técnico ou administrativo, como nomes de autores, contatos institucionais, fichas catalográficas, endereços e notas editoriais.

**O que o Script Faz**
O script executa as seguintes tarefas:

- Lê o arquivo `paragrafos_validos_corpus.xlsx`, que contém todos os parágrafos extraídos dos arquivos .txt considerados válidos estruturalmente.
- Aplica uma função de filtragem semântica (`paragrafo_relevante`):
    - Remove parágrafos curtos (com menos de 6 palavras).
    - Exclui parágrafos que contenham padrões de texto como:
        - Nomes de autores e responsáveis técnicos;
        - Endereços e e-mails;
        - Informações de publicação (tiragem, edição, versão);
        - Trechos genéricos como “anexo”, “índice”, “apêndice”;
        - Informações institucionais repetitivas (ex: “Ministério da Saúde”).

**Cria dois novos arquivos:** <img src="https://img.icons8.com/ios-filled/50/000000/document--v1.png" width="24" style="vertical-align:middle; margin-left:8px;"/>

*paragrafos_filtrados_corpus.xlsx: apenas com os parágrafos considerados relevantes.

relatorio_filtragem_paragrafos.xlsx: com estatísticas da filtragem (totais e percentuais) e amostras dos parágrafos excluídos e mantidos.*

In [1]:
import pandas as pd
import re
import os

In [23]:
# === CONFIGURAÇÕES DE CAMINHO ===
PASTA = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
ARQUIVO_ENTRADA = os.path.join(PASTA, "paragrafos_validos_corpus.xlsx")
ARQUIVO_SAIDA = os.path.join(PASTA, "paragrafos_filtrados_corpus.xlsx")
ARQUIVO_RELATORIO = os.path.join(PASTA, "relatorio_filtragem_paragrafos.xlsx")

# === FUNÇÃO DE FILTRAGEM ===
def paragrafo_relevante(texto):
    texto_limpo = texto.lower().strip()
    padroes_excluir = [
        r'nomes?[:\-]',
        r'e-mail|site:|www\.',
        r'cep:|quadra|srtv|endere',
        r'coordenação-geral|departamento',
        r'ministério da saúde',
        r'esta obra.*licença',
        r'(nome|cargo) do autor',
        r'saúde pública brasileira.*acesso',
        r'\b(anexo|apêndice|índice)\b',
        r'(autores?|versão|tiragem|edição).*202[0-9]',
        r'(sigla|significado) de ',
        r'(organizador|responsável|revisão)',
    ]
    for padrao in padroes_excluir:
        if re.search(padrao, texto_limpo):
            return False
    if len(texto_limpo.split()) < 6:
        return False
    return True

# === EXECUÇÃO ===
print("🔍 Lendo arquivo...")
df = pd.read_excel(ARQUIVO_ENTRADA)

print("⚙️ Aplicando filtro semântico...")
df["Relevante"] = df["Texto"].astype(str).apply(paragrafo_relevante)

df_filtrado = df[df["Relevante"]].drop(columns=["Relevante"])
df_excluido = df[~df["Relevante"]].drop(columns=["Relevante"])

print(f"✅ Parágrafos mantidos: {len(df_filtrado)}")
print(f"🚫 Parágrafos excluídos: {len(df_excluido)}")

# === SALVAR RESULTADO FILTRADO ===
print("💾 Salvando arquivo com parágrafos filtrados...")
df_filtrado.to_excel(ARQUIVO_SAIDA, index=False)

# === GERAR RELATÓRIO EM EXCEL COM AMOSTRAS ===
print("📝 Gerando relatório...")
resumo = {
    "Total original": [len(df)],
    "Total mantido": [len(df_filtrado)],
    "Total excluído": [len(df_excluido)],
    "% mantido": [round(100 * len(df_filtrado) / len(df), 2)],
    "% excluído": [round(100 * len(df_excluido) / len(df), 2)],
}
df_resumo = pd.DataFrame(resumo)

amostra_mantido = df_filtrado.sample(min(5, len(df_filtrado)), random_state=1)
amostra_excluido = df_excluido.sample(min(5, len(df_excluido)), random_state=1)

with pd.ExcelWriter(ARQUIVO_RELATORIO) as writer:
    df_resumo.to_excel(writer, sheet_name="Resumo", index=False)
    amostra_mantido.to_excel(writer, sheet_name="Amostra Mantido", index=False)
    amostra_excluido.to_excel(writer, sheet_name="Amostra Excluído", index=False)

print("📊 Relatório salvo com sucesso!")
print(f"📁 Arquivo filtrado: {ARQUIVO_SAIDA}")
print(f"📄 Relatório: {ARQUIVO_RELATORIO}")


🔍 Lendo arquivo...


TypeError: string argument expected, got 'ExpatError'

TypeError: string argument expected, got 'ExpatError'

In [ ]:
#diagnosticar_pasta Antes da Etapa 2 – Extração e limpeza de texto Diagnóstico visual de duplicados e nomes corrompidos
#COMEÇA AQUI

import os
import hashlib
import re
import csv
from pathlib import Path
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext

# === FUNÇÕES AUXILIARES ===

def calcular_hash(caminho_arquivo, limite=1024*1024):
    """Calcula hash de uma parte do arquivo (1 MB por padrão) para diagnóstico rápido."""
    sha256 = hashlib.sha256()
    with open(caminho_arquivo, 'rb') as f:
        sha256.update(f.read(limite))
    return sha256.hexdigest()

def normalizar_nome(nome):
    """Remove repetições consecutivas de sufixos."""
    padrao = r'(txt_\w+\d{4})+'
    return re.sub(padrao, lambda m: m.group(1), nome)

def diagnosticar_pasta(pasta_raiz, saida_texto):
    pasta_raiz = Path(pasta_raiz)
    hash_map = {}
    log_linhas = []
    log_path = pasta_raiz / "log_diagnostico.csv"

    saida_texto.insert(tk.END, f"🔍 Diagnóstico em: {pasta_raiz}\n")
    saida_texto.insert(tk.END, "Apenas leitura — nenhum arquivo será alterado.\n\n")
    saida_texto.update()

    for caminho_arquivo in pasta_raiz.rglob("*.txt"):
        hash_arquivo = calcular_hash(caminho_arquivo)
        nome_original = caminho_arquivo.name
        caminho_pasta = caminho_arquivo.parent

        # Verifica duplicidade de conteúdo
        if hash_arquivo in hash_map:
            saida_texto.insert(tk.END, f"⚠️ Possível duplicado: {nome_original} (igual a {hash_map[hash_arquivo].name})\n")
            log_linhas.append(["DUPLICADO", str(caminho_arquivo), "Mesma hash de", str(hash_map[hash_arquivo])])
        else:
            hash_map[hash_arquivo] = caminho_arquivo

        # Verifica nome corrompido
        nome_corrigido = normalizar_nome(nome_original)
        if nome_corrigido != nome_original:
            saida_texto.insert(tk.END, f"🔁 Nome suspeito: {nome_original} → Sugerido: {nome_corrigido}\n")
            log_linhas.append(["NOME_CORROMPIDO", str(caminho_arquivo), "Sugerido", nome_corrigido])

    with open(log_path, mode='w', newline='', encoding='utf-8') as log_csv:
        writer = csv.writer(log_csv)
        writer.writerow(["Tipo", "Arquivo", "Info", "Referência/Sugestão"])
        writer.writerows(log_linhas)

    saida_texto.insert(tk.END, f"\n📄 Log salvo em: {log_path}\n")
    saida_texto.insert(tk.END, "✅ Diagnóstico concluído.\n")
    saida_texto.update()

# === INTERFACE GRÁFICA ===

def iniciar_interface():
    def escolher_pasta():
        pasta = filedialog.askdirectory(title="Selecione a pasta 'corpus_final'")
        if pasta:
            entrada_pasta.delete(0, tk.END)
            entrada_pasta.insert(0, pasta)

    def executar_diagnostico():
        pasta_escolhida = entrada_pasta.get()
        if not os.path.isdir(pasta_escolhida):
            messagebox.showerror("Erro", "Selecione uma pasta válida.")
            return
        saida_texto.delete(1.0, tk.END)
        diagnosticar_pasta(pasta_escolhida, saida_texto)

    janela = tk.Tk()
    janela.title("🔎 Diagnóstico de Arquivos Duplicados e Nomes Corrompidos")

    tk.Label(janela, text="📁 Pasta Base:").pack(pady=5)
    entrada_pasta = tk.Entry(janela, width=60)
    entrada_pasta.pack()
    tk.Button(janela, text="Selecionar Pasta", command=escolher_pasta).pack(pady=5)

    tk.Button(janela, text="🔍 Executar Diagnóstico", command=executar_diagnostico, bg="blue", fg="white").pack(pady=10)

    saida_texto = scrolledtext.ScrolledText(janela, width=100, height=25)
    saida_texto.pack(padx=10, pady=10)

    janela.mainloop()

# Rodar
if __name__ == "__main__":
    iniciar_interface()


In [11]:
#executar_log_diagnostico remove duplicatas reais e renomeia arquivos com nome corrompido.Logo após o diagnóstico, ainda na Etapa 2
import os
import csv
from pathlib import Path

# === CONFIGURAÇÃO ===
PASTA_BASE = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")  # ajuste se necessário
CAMINHO_LOG_DIAGNOSTICO = PASTA_BASE / "log_diagnostico.csv"
CAMINHO_LOG_EXECUCAO = PASTA_BASE / "log_execucao.csv"

# === LEITURA DO LOG ===
with open(CAMINHO_LOG_DIAGNOSTICO, encoding='utf-8') as f:
    leitor = csv.DictReader(f)
    acoes = list(leitor)

# === EXECUÇÃO DAS AÇÕES ===
log_execucao = []

for linha in acoes:
    tipo = linha['Tipo']
    caminho_str = linha['Arquivo']
    caminho_arquivo = Path(caminho_str)

    if not caminho_arquivo.exists():
        log_execucao.append(["IGNORADO", caminho_str, "Arquivo não encontrado"])
        continue

    if tipo == "DUPLICADO":
        try:
            caminho_arquivo.unlink()
            log_execucao.append(["REMOVIDO", caminho_str, "Duplicata removida"])
            print(f"🗑️ Removido: {caminho_str}")
        except Exception as e:
            log_execucao.append(["ERRO", caminho_str, f"Erro ao remover: {e}"])

    elif tipo == "NOME_CORROMPIDO":
        sugestao = linha['Referência/Sugestão']
        novo_caminho = caminho_arquivo.parent / sugestao
        contador = 1
        while novo_caminho.exists():
            novo_caminho = caminho_arquivo.parent / f"{sugestao}_{contador}.txt"
            contador += 1
        try:
            caminho_arquivo.rename(novo_caminho)
            log_execucao.append(["RENOMEADO", caminho_str, f"Novo nome: {novo_caminho.name}"])
            print(f"✏️ Renomeado: {caminho_arquivo.name} → {novo_caminho.name}")
        except Exception as e:
            log_execucao.append(["ERRO", caminho_str, f"Erro ao renomear: {e}"])

# === SALVA LOG FINAL ===
with open(CAMINHO_LOG_EXECUCAO, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Ação", "Arquivo", "Detalhes"])
    writer.writerows(log_execucao)

print(f"\n✅ Execução concluída. Log salvo em: {CAMINHO_LOG_EXECUCAO}")


✅ Execução concluída. Log salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\log_execucao.csv


In [21]:
import os
import pandas as pd
from pathlib import Path
import re
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

# === CONFIGURAÇÃO ===
PASTA_CORPUS = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")
ARQUIVO_SAIDA = PASTA_CORPUS / "paragrafos_validos_corpus.xlsx"
ARQUIVO_DIAGNOSTICO = PASTA_CORPUS / "diagnostico_paragrafos_por_arquivo.xlsx"

# === FUNÇÕES ===
def paragrafo_valido(paragrafo):
    paragrafo = paragrafo.strip()
    if len(paragrafo.split()) < 5:
        return False
    if re.match(r'^(\d+|[•*])$', paragrafo):  # apenas número ou bullet
        return False
    if sum(1 for palavra in ["ministério", "licença", "tiragem", "elaboração", "cep", "autores", "versão", "editoração"] if palavra in paragrafo.lower()) >= 2:
        return False
    return True

def limpar_caracteres_invalidos(texto):
    return ILLEGAL_CHARACTERS_RE.sub("", texto)

def quebrar_paragrafos_brutos(texto):
    # Substitui quebras de linha simples por espaço (evita quebra errada)
    texto = texto.replace("\n", " ")
    # Insere quebra forçada após ponto final seguido de espaço e letra maiúscula
    texto = re.sub(r'\. (?=[A-ZÁÉÍÓÚÇ])', '.\n\n', texto)
    # Divide onde houver \n\n forçado
    return [p.strip() for p in texto.split('\n\n') if p.strip()]

# === PROCESSAMENTO ===
dados = []
diagnostico = []

for caminho_txt in PASTA_CORPUS.rglob("*.txt"):
    try:
        with open(caminho_txt, "r", encoding="utf-8") as f:
            conteudo = f.read()

        # Divide usando segmentação forçada
        paragrafos_raw = quebrar_paragrafos_brutos(conteudo)
        paragrafos_validos = [p for p in paragrafos_raw if paragrafo_valido(p)]

        for i, paragrafo in enumerate(paragrafos_validos, 1):
            dados.append({
                "Arquivo": caminho_txt.name,
                "Parágrafo": limpar_caracteres_invalidos(paragrafo),
                "Número do parágrafo": i
            })

        diagnostico.append({
            "Arquivo": caminho_txt.name,
            "Total lidos": len(paragrafos_raw),
            "Válidos mantidos": len(paragrafos_validos),
            "Descartados": len(paragrafos_raw) - len(paragrafos_validos)
        })

    except Exception as e:
        print(f"⚠️ Erro ao processar {caminho_txt.name}: {e}")

# === EXPORTAÇÃO ===
df_paragrafos = pd.DataFrame(dados)
df_paragrafos.to_excel(ARQUIVO_SAIDA, index=False)

df_diag = pd.DataFrame(diagnostico)
df_diag.to_excel(ARQUIVO_DIAGNOSTICO, index=False)

ARQUIVO_CSV = PASTA_CORPUS / "paragrafos_validos_corpus.csv"
df_paragrafos.to_csv(ARQUIVO_CSV, index=False, encoding="utf-8")
print(f"📄 Parágrafos válidos salvos em {ARQUIVO_SAIDA}")
print(f"✅ {len(df_paragrafos)} parágrafos válidos salvos em {ARQUIVO_SAIDA}")
print(f"🩺 Diagnóstico salvo em {ARQUIVO_DIAGNOSTICO}")





📄 Parágrafos válidos salvos em C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\paragrafos_validos_corpus.xlsx
✅ 20106 parágrafos válidos salvos em C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\paragrafos_validos_corpus.xlsx
🩺 Diagnóstico salvo em C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\diagnostico_paragrafos_por_arquivo.xlsx


In [22]:
import pandas as pd
from pathlib import Path

# === CONFIGURAÇÃO ===
PASTA_BASE = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")
ARQUIVO_ENTRADA = PASTA_BASE / "resumo_corpus_final.csv"
ARQUIVO_UNICOS = PASTA_BASE / "resumo_arquivos_unicos.xlsx"
ARQUIVO_AGREGADO = PASTA_BASE / "resumo_por_subpasta.xlsx"

# === LEITURA DO ARQUIVO DE RESUMO ===
df = pd.read_csv(ARQUIVO_ENTRADA)

# === REMOVER DUPLICATAS POR NOME BASE DO ARQUIVO ===
# Extrai o nome base (sem partes repetidas)
df["Arquivo_base"] = df["Arquivo"].str.extract(r"^(.+?)(?:txt|\.txt)", expand=False).str.strip()
df_unicos = df.drop_duplicates(subset=["Subpasta", "Arquivo_base"])

# === AGRUPAMENTO POR SUBPASTA ===
df_grupo = df_unicos.groupby("Subpasta").agg({
    "Arquivo": "count",
    "Tamanho (bytes)": "sum",
    "Nº de linhas": "mean",
    "Nº de caracteres": "mean"
}).reset_index()

df_grupo = df_grupo.rename(columns={
    "Arquivo": "Total de arquivos únicos",
    "Tamanho (bytes)": "Tamanho total (bytes)",
    "Nº de linhas": "Média de linhas",
    "Nº de caracteres": "Média de caracteres"
})

# === EXPORTAÇÃO ===
df_unicos.to_excel(ARQUIVO_UNICOS, index=False)
df_grupo.to_excel(ARQUIVO_AGREGADO, index=False)

print(f"✅ Arquivo único salvo em: {ARQUIVO_UNICOS}")
print(f"📊 Resumo por subpasta salvo em: {ARQUIVO_AGREGADO}")


ERROR! Session/line number was not unique in database. History logging moved to new session 39
✅ Arquivo único salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\resumo_arquivos_unicos.xlsx
📊 Resumo por subpasta salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\resumo_por_subpasta.xlsx


In [ ]:
####Geração de corpus unificado (`corpus_unificado.txt`)
#Após as etapas de extração e filtragem dos textos normativos, é gerado um arquivo final contendo todos os `.txt` limpos, organizados por subpasta e nome de arquivo.
# O texto abaixo é apenas explicativo, não executável.
# Esta etapa permite ao usuário validar rapidamente o corpus unificado, inspecionando o conteúdo final dos arquivos .txt já limpos e organizados.
# O arquivo corpus_unificado.txt pode ser aberto em qualquer editor de texto para conferência visual, busca de termos, ou revisão por especialistas.


import os
from pathlib import Path

# === CONFIGURAÇÕES ===
PASTA_CORPUS = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")
ARQUIVO_SAIDA = PASTA_CORPUS / "corpus_unificado.txt"

# === UNIFICAR CONTEÚDO ===
with open(ARQUIVO_SAIDA, "w", encoding="utf-8") as arquivo_saida:
    for caminho_txt in PASTA_CORPUS.rglob("*.txt"):
        try:
            with open(caminho_txt, "r", encoding="utf-8") as f:
                conteudo = f.read().strip()

            if len(conteudo) < 50:
                continue  # ignora arquivos muito curtos

            nome_arquivo = caminho_txt.name
            subpasta = caminho_txt.parent.name

            bloco = f"\n\n### Arquivo: {nome_arquivo} | Subpasta: {subpasta}\n\n{conteudo}\n\n"
            arquivo_saida.write(bloco)

        except Exception as e:
            print(f"Erro ao ler {caminho_txt.name}: {e}")

print(f"✅ Corpus unificado salvo em: {ARQUIVO_SAIDA}")


✅ Corpus unificado salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\corpus_unificado.txt


In [27]:
import pandas as pd
import re
import os
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

# === CAMINHOS ===
PASTA = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
ARQUIVO_ENTRADA = os.path.join(PASTA, "paragrafos_validos_corpus.csv")
ARQUIVO_SAIDA = os.path.join(PASTA, "paragrafos_filtrados_corpus.csv")
ARQUIVO_RELATORIO = os.path.join(PASTA, "relatorio_filtragem_paragrafos.csv")

# === FILTRO SEMÂNTICO COM PROTEÇÕES RÍGIDAS ===
def paragrafo_relevante(texto):
    texto_limpo = str(texto).strip()

    if len(texto_limpo) > 3000:  # ainda mais seguro
        return False
    if ILLEGAL_CHARACTERS_RE.search(texto_limpo):
        return False

    texto_limpo = texto_limpo.lower()
    padroes_excluir = [
        r'nomes?[:\-]',
        r'e-mail|site:|www\.',
        r'cep:|quadra|srtv|endere',
        r'coordenação-geral|departamento',
        r'ministério da saúde',
        r'esta obra.*licença',
        r'(nome|cargo) do autor',
        r'saúde pública brasileira.*acesso',
        r'\b(anexo|apêndice|índice)\b',
        r'(autores?|versão|tiragem|edição).*202[0-9]',
        r'(sigla|significado) de ',
        r'(organizador|responsável|revisão)',
    ]
    for padrao in padroes_excluir:
        if re.search(padrao, texto_limpo):
            return False
    if len(texto_limpo.split()) < 6:
        return False
    return True

# === LEITURA ===
df = pd.read_csv(ARQUIVO_ENTRADA)
df["Relevante"] = df["Parágrafo"].astype(str).apply(paragrafo_relevante)

# === SEPARAÇÃO ===
df_filtrado = df[df["Relevante"]].drop(columns=["Relevante"])
df_excluido = df[~df["Relevante"]].drop(columns=["Relevante"])

# === SALVAR RESULTADOS EM .CSV ===
df_filtrado.to_csv(ARQUIVO_SAIDA, index=False, encoding="utf-8")
df_excluido.to_csv(PASTA + "/paragrafos_descartados.csv", index=False, encoding="utf-8")

# === RELATÓRIO RESUMIDO ===
resumo = pd.DataFrame([{
    "Total original": len(df),
    "Total mantido": len(df_filtrado),
    "Total excluído": len(df_excluido),
    "% mantido": round(100 * len(df_filtrado) / len(df), 2),
    "% excluído": round(100 * len(df_excluido) / len(df), 2),
}])
resumo.to_csv(ARQUIVO_RELATORIO, index=False)

print(f"✅ CSV salvo com {len(df_filtrado)} parágrafos: {ARQUIVO_SAIDA}")
print(f"📊 Relatório salvo: {ARQUIVO_RELATORIO}")



✅ CSV salvo com 17640 parágrafos: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\paragrafos_filtrados_corpus.csv
📊 Relatório salvo: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\relatorio_filtragem_paragrafos.csv


In [32]:
# Instalação dos pacotes necessários
%pip install torch transformers tqdm sentencepiece

import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datetime import datetime
import os
from tqdm import tqdm

# === CONFIGURAÇÕES ===
PASTA = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
ARQUIVO_ENTRADA = os.path.join(PASTA, "paragrafos_filtrados_corpus.csv")
DATA = datetime.today().strftime('%Y%m%d')
ARQUIVO_SAIDA = os.path.join(PASTA, f"perguntas_geradas_{DATA}.csv")

# === CARREGAR MODELO T5 DE GERAÇÃO DE PERGUNTAS ===
modelo = "valhalla/t5-base-qg-hl"
tokenizer = T5Tokenizer.from_pretrained(modelo)
model = T5ForConditionalGeneration.from_pretrained(modelo)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# === FUNÇÃO PARA GERAR PERGUNTA COM TRATAMENTO DE ERRO ===
def gerar_pergunta(texto):
    try:
        entrada = f"generate question: {texto}"
        inputs = tokenizer(entrada, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(inputs["input_ids"], max_length=64)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as e:
        return f"[ERRO] {str(e)}"

# === LEITURA DO CSV ===
df = pd.read_csv(ARQUIVO_ENTRADA)

# (Opcional) limitar para teste:
# df = df.head(50)

# === GERAÇÃO DAS PERGUNTAS COM BARRA DE PROGRESSO ===
tqdm.pandas()
df["Pergunta"] = df["Parágrafo"].astype(str).progress_apply(gerar_pergunta)

# === SALVAR SAÍDA ===
df.to_csv(ARQUIVO_SAIDA, index=False, encoding="utf-8")
print(f"✅ Perguntas geradas salvas em: {ARQUIVO_SAIDA}")



   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   --------------------------------------- 992.0/992.0 kB 11.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [32]:
#filtro_paragrafos_relevantes Filtra semanticamente os parágrafos
import pandas as pd
import os
import re
from collections import Counter
from datetime import datetime

# === CAMINHOS ===
PASTA_CORPUS = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
ARQUIVO_PARAGRAFOS = os.path.join(PASTA_CORPUS, "paragrafos_validos_corpus.xlsx")
ARQUIVO_SAIDA = os.path.join(PASTA_CORPUS, f"perguntas_geradas_{datetime.today().strftime('%Y%m%d')}.xlsx")

# === FUNÇÃO PARA GERAR PERGUNTAS COMUNS ===
def gerar_perguntas(texto):
    perguntas = []

    # Baixo nível de NLP, apenas heurístico (foco em utilidade prática)
    if re.search(r"\bprocedimento[s]?\b", texto, re.IGNORECASE):
        perguntas.append("Quais são os procedimentos recomendados?")
    if re.search(r"\bexame[s]?\b", texto, re.IGNORECASE):
        perguntas.append("Como é realizado o exame?")
    if re.search(r"\btratamento\b", texto, re.IGNORECASE):
        perguntas.append("Qual é o tratamento indicado?")
    if re.search(r"\bnotifica(ção|r)\b", texto, re.IGNORECASE):
        perguntas.append("Quando o caso deve ser notificado?")
    if re.search(r"\bmedicamento[s]?\b", texto, re.IGNORECASE):
        perguntas.append("Quais medicamentos devem ser utilizados?")
    if re.search(r"\bsintoma[s]?\b", texto, re.IGNORECASE):
        perguntas.append("Quais são os sintomas?")
    if re.search(r"\bmonitoramento\b", texto, re.IGNORECASE):
        perguntas.append("Como é feito o monitoramento?")
    if re.search(r"\bvigilância\b", texto, re.IGNORECASE):
        perguntas.append("Quais são as ações de vigilância indicadas?")
    if re.search(r"\bsurto[s]?\b", texto, re.IGNORECASE):
        perguntas.append("O que deve ser feito em caso de surto?")

    return perguntas

# === CARGA DE PARÁGRAFOS ===
df = pd.read_excel(ARQUIVO_PARAGRAFOS)

# === GERAR PERGUNTAS POR PARÁGRAFO ===
linhas_saida = []

for _, row in df.iterrows():
    arquivo = row["Arquivo"]
    texto = str(row["Texto"])
    perguntas = gerar_perguntas(texto)

    for pergunta in perguntas:
        linhas_saida.append({
            "Arquivo (Tema)": arquivo,
            "Pergunta sugerida": pergunta,
            "Texto de origem": texto
        })

# === SALVAR RESULTADO ===
df_resultado = pd.DataFrame(linhas_saida)
df_resultado.to_excel(ARQUIVO_SAIDA, index=False)

print("✅ Perguntas geradas com sucesso!")
print(f"📄 Arquivo salvo em: {ARQUIVO_SAIDA}")


TypeError: string argument expected, got 'ExpatError'

TypeError: string argument expected, got 'ExpatError'

In [14]:
# === TESTE COM AMOSTRA DE 100 PARÁGRAFOS ===
df = pd.read_excel(ARQUIVO_PARAGRAFOS).sample(100, random_state=42)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [33]:
print(df.head(100))

                                 Arquivo  \
0   adenovirus_guia_1txt_guiavsa2023.txt   
1   adenovirus_guia_1txt_guiavsa2023.txt   
2   adenovirus_guia_1txt_guiavsa2023.txt   
3   adenovirus_guia_1txt_guiavsa2023.txt   
4   adenovirus_guia_1txt_guiavsa2023.txt   
..                                   ...   
95  adenovirus_guia_1txt_guiavsa2023.txt   
96  adenovirus_guia_1txt_guiavsa2023.txt   
97  adenovirus_guia_1txt_guiavsa2023.txt   
98  adenovirus_guia_1txt_guiavsa2023.txt   
99  adenovirus_guia_1txt_guiavsa2023.txt   

                                            Parágrafo  Número do parágrafo  \
0   1 VIGILÂNCIA INTEGRADA DA COVID-19, INFLUENZA ...                    1   
1   Diversas famílias de vírus respiratórios estão...                    2   
2   Os vírus influenza, vírus sincicial respiratór...                    3   
3   A vigilância de influenza foi implantada em 20...                    4   
4   Pela característica sindrômica da doença, outr...                    5   
.. 

ERROR! Session/line number was not unique in database. History logging moved to new session 44


In [35]:
#gerador_perguntas_t5
# === CONTINUAR AQUI COM O MESMO SCRIPT ===
linhas = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Gerando perguntas - Amostra"):
    texto = str(row.get("Texto", "")).strip()
    if not texto:
        continue
    try:
        perguntas = gerar_perguntas_t5(texto)
        for pergunta in perguntas:
            linhas.append({
                "Arquivo (Tema)": row.get("Arquivo", ""),
                "Pergunta sugerida": pergunta,
                "Texto de origem": texto
            })
    except Exception as e:
        print(f"⚠️ Erro na linha {idx}: {e}")

# === SALVAR RESULTADO FINAL ===
ARQUIVO_TESTE = os.path.join(PASTA_CORPUS, "perguntas_amostra.xlsx")
df_resultado = pd.DataFrame(linhas)
df_resultado.to_excel(ARQUIVO_TESTE, index=False)

print("✅ Perguntas da amostra geradas com sucesso!")
print(f"📄 Arquivo salvo em: {ARQUIVO_TESTE}")


Gerando perguntas - Amostra: 100%|██████████| 20106/20106 [00:00<00:00, 40881.95it/s]

✅ Perguntas da amostra geradas com sucesso!
📄 Arquivo salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_amostra.xlsx


In [36]:
# Instalação dos pacotes necessários (pode deixar só na primeira célula do notebook)
%pip install transformers torch pandas openpyxl tqdm sentencepiece

# === IMPORTAÇÕES ===
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datetime import datetime
from tqdm import tqdm
import os

# === CONFIGURAÇÃO DE CAMINHOS ===
PASTA_CORPUS = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
ARQUIVO_PARAGRAFOS = os.path.join(PASTA_CORPUS, "paragrafos_validos_corpus.xlsx")
ARQUIVO_SAIDA = os.path.join(PASTA_CORPUS, f"perguntas_avancadas_{datetime.today().strftime('%Y%m%d')}.xlsx")

# === CARREGAR MODELO E TOKENIZER ===
MODELO = "valhalla/t5-base-qg-hl"
tokenizer = T5Tokenizer.from_pretrained(MODELO)
model = T5ForConditionalGeneration.from_pretrained(MODELO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# === FUNÇÃO DE GERAÇÃO DE PERGUNTAS ===
def gerar_perguntas_t5(texto):
    entrada = f"generate question: {texto} </s>"
    input_ids = tokenizer.encode(entrada, return_tensors="pt").to(device)
    with torch.no_grad():  # otimiza uso de memória
        saida_ids = model.generate(
            input_ids=input_ids,
            max_length=64,
            num_beams=4,
            num_return_sequences=3,
            early_stopping=True
        )
    perguntas = [tokenizer.decode(g, skip_special_tokens=True) for g in saida_ids]
    return list(set(perguntas))  # remove duplicadas

# === CARREGAR PARÁGRAFOS ===
df = pd.read_excel(ARQUIVO_PARAGRAFOS)
linhas = []

# === PERCORRER PARÁGRAFOS COM BARRA DE PROGRESSO ===
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Gerando perguntas"):
    texto = str(row.get("Texto", "")).strip()
    if not texto:
        continue
    try:
        perguntas = gerar_perguntas_t5(texto)
        for pergunta in perguntas:
            linhas.append({
                "Arquivo (Tema)": row.get("Arquivo", ""),
                "Pergunta sugerida": pergunta,
                "Texto de origem": texto
            })
        # Checkpoint: salva a cada 100 parágrafos
        if idx % 100 == 0 and idx != 0:
            pd.DataFrame(linhas).to_excel(ARQUIVO_SAIDA, index=False)
    except Exception as e:
        print(f"⚠️ Erro na linha {idx}: {e}")

# === SALVAR RESULTADO FINAL ===
df_resultado = pd.DataFrame(linhas)
df_resultado.to_excel(ARQUIVO_SAIDA, index=False)

print("✅ Perguntas geradas com sucesso!")
print(f"📄 Arquivo salvo em: {ARQUIVO_SAIDA}")

Note: you may need to restart the kernel to use updated packages.


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
